<a href="https://colab.research.google.com/github/BATUTO90/Bat1/blob/main/Proyecto_multifuncional_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

os.environ["SAMBANOVA_API_KEY"] = userdata.get("SAMBANOVA_API_KEY")

# Opcional: para Hugging Face si lo tienes guardado
os.environ["HUGGINGFACE_TOKEN"] = userdata.get("HUGGINGFACE_TOKEN")

In [ ]:
pip install agentops python-dotenv

In [ ]:
AGENTOPS_API_KEY= "5e68f37b-36a7-443a-8453-f88fe3923213"

In [ ]:
%%bash
export AGENTOPS_API_KEY="5e68f37b-36a7-443a-8453-f88fe3923213"

In [ ]:
%pip install sambanova

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
%%writefile requirements.txt
sambanova==1.1.1
transformers
torch
gradio
python-dotenv
sentence-transformers
agentops
litellm
tqdm
pymilvus
requests

Writing requirements.txt


In [ ]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 27.2 MB/s eta 0:00:00
  Attempting uninstall: sambanova
    Found existing installation: sambanova 1.1.5
    Uninstalling sambanova-1.1.5:
      Successfully uninstalled sambanova-1.1.5


In [ ]:
import os
import gradio as gr
import asyncio
import torch
from PIL import Image
from io import BytesIO
import base64
import json
from sambanova import SambaNova
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler  # Agregado para scheduler optimizado
from functools import lru_cache
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Inicializar AgentOps solo si está configurado
try:
    import agentops
    AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")
    if not AGENTOPS_API_KEY:
        raise ValueError("Falta AGENTOPS_API_KEY")
    agentops.init(AGENTOPS_API_KEY, tags=["multimodal_assistant"], trace_name="MainSession")
    AGENTOPS_ENABLED = True
except (ImportError, ValueError):
    AGENTOPS_ENABLED = False

# Claves API SambaNova y Hugging Face desde entorno
SAMBANOVA_API_KEY = os.getenv("SAMBANOVA_API_KEY")
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")  # Usar HF_TOKEN para HuggingFace

if not SAMBANOVA_API_KEY:
    raise ValueError("❌ Debe configurar SAMBANOVA_API_KEY")

sn_client = SambaNova(api_key=SAMBANOVA_API_KEY, base_url="https://api.sambanova.ai/v1")

MODELS = {
    "chat": "Llama-3.3-Swallow-70B-Instruct-v0.4",
    "qwen": "Qwen3-32B",
    "code": "bigcode/starcoder2-7b",
    "llama_meta": "Meta-Llama-3.1-8B-Instruct",
    "llama_maverick": "Llama-4-Maverick-17B-128E-Instruct"
}

# Optimizaciones globales para CPU (para Stable Diffusion)
torch.backends.cudnn.benchmark = False
if torch.__version__ >= '2.0':
    torch.backends.cudnn.allow_tf32 = False

# Cargar modelo local de Hugging Face para código
class ModelLoader:
    @lru_cache(maxsize=2)
    def load_model(self, model_name):
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=HUGGINGFACE_TOKEN)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            use_auth_token=HUGGINGFACE_TOKEN,
            device_map="auto" if torch.cuda.is_available() else {"": "cpu"},
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        )
        model.eval()
        return model, tokenizer

model_loader = ModelLoader()

# Cargar pipeline diffusion optimizado para CPU: modelo v1-5 con FP16 y scheduler rápido
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",  # Modelo más eficiente que v1-4
    torch_dtype=torch.float16,  # FP16 para reducir computo en CPU
    use_safetensors=True,  # Más rápido y seguro
    use_auth_token=HUGGINGFACE_TOKEN,
    variant="fp16"  # Carga variante half-precision si disponible
)

# Deshabilita safety checker para ahorrar tiempo
pipe.safety_checker = None
pipe.requires_safety_checker = False

# Scheduler más rápido: DPMSolverMultistep (pocos pasos para calidad/velocidad)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Mover a CPU (o CUDA si disponible)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

# Opcional: Compilación con Torch 2.0+ para speedup (~20%)
if torch.__version__ >= '2.0' and device == "cpu":
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)


def image_to_data_url(img: Image.Image) -> str:
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_b64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{img_b64}"


# Flag para precalentamiento
generate_image.warmed_up = False

async def generate_image(prompt: str, num_inference_steps=25, width=512, height=512, guidance_scale=7.5):
    loop = asyncio.get_event_loop()

    def sync_generate():
        nonlocal generate_image
        # Precalienta si es la primera vez (solo una vez)
        if not generate_image.warmed_up:
            _ = pipe("dummy prompt", num_inference_steps=1).images[0]
            generate_image.warmed_up = True

        # Genera con opts optimizadas para CPU
        image = pipe(
            prompt,
            num_inference_steps=num_inference_steps,  # 25 pasos: balance calidad/velocidad
            guidance_scale=guidance_scale,  # 7.5 para buena adherencia al prompt
            width=width,
            height=height,
            generator=torch.Generator(device).manual_seed(42)  # Seed fijo
        ).images[0]
        return image

    image = await loop.run_in_executor(None, sync_generate)
    return image


async def generate_sambanova(prompt: str, model_name: str):
    loop = asyncio.get_event_loop()
    if AGENTOPS_ENABLED:
        tracer = agentops.start_trace(trace_name=f"SambaNova-{model_name}")
    try:
        res = await loop.run_in_executor(
            None,
            lambda: sn_client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.6,
                top_p=0.9,
                max_tokens=500
            )
        )
        return res.choices[0].message.content
    finally:
        if AGENTOPS_ENABLED:
            agentops.end_trace(tracer, end_state="Success")


async def generate_llama_maverick_with_image(prompt: str, img: Image.Image):
    loop = asyncio.get_event_loop()
    data_url = image_to_data_url(img)
    message_content = [
        {"type": "text", "text": prompt},
        {"type": "image_url", "image_url": {"url": data_url}}
    ]
    if AGENTOPS_ENABLED:
        tracer = agentops.start_trace(trace_name="LlamaMaverickImage")
    try:
        res = await loop.run_in_executor(
            None,
            lambda: sn_client.chat.completions.create(
                model=MODELS["llama_maverick"],
                messages=[{"role": "user", "content": message_content}],
                temperature=0.1,
                top_p=0.1
            )
        )
        return res.choices[0].message.content
    finally:
        if AGENTOPS_ENABLED:
            agentops.end_trace(tracer, end_state="Success")


async def generate_code(prompt: str):
    model, tokenizer = model_loader.load_model(MODELS["code"])
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.4,
            pad_token_id=tokenizer.eos_token_id
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


async def get_current_weather(location, unit="fahrenheit"):
    # Dummy function example
    if unit == "fahrenheit":
        return {"location": location, "temperature": "72", "unit": "fahrenheit"}
    else:
        return {"location": location, "temperature": "22", "unit": "celsius"}


tools = {
    "get_current_weather": get_current_weather
}


def convert_to_messages_format(history):
    """Convertir historial de tuplas a formato de mensajes para Gradio Chatbot type='messages'."""
    messages = []
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if isinstance(assistant_msg, tuple):
            # Para respuestas con imagen: (texto, imagen)
            content = [{"type": "text", "text": assistant_msg[0]}]
            if len(assistant_msg) > 1 and assistant_msg[1] is not None:
                img_data_url = image_to_data_url(assistant_msg[1])
                content.append({"type": "image_url", "image_url": {"url": img_data_url}})
            messages.append({"role": "assistant", "content": content})
        else:
            messages.append({"role": "assistant", "content": str(assistant_msg)})
    return messages


async def process_input(message, image, history):
    # Convertir historial entrante al formato de tuplas si es necesario (para compatibilidad interna)
    if isinstance(history, list) and history and isinstance(history[0], dict):
        # Si entra como messages, convertir a tuplas para procesamiento
        tuple_history = []
        i = 0
        while i < len(history):
            if history[i]["role"] == "user":
                user_msg = history[i]["content"] if isinstance(history[i]["content"], str) else history[i]["content"][0]["text"] if history[i]["content"] else ""
                i += 1
                if i < len(history) and history[i]["role"] == "assistant":
                    assistant_msg = history[i]["content"]
                    if isinstance(assistant_msg, list):
                        # Si es multimodal, extraer texto principal
                        text_parts = [part["text"] for part in assistant_msg if part["type"] == "text"]
                        assistant_msg = " ".join(text_parts) if text_parts else str(assistant_msg)
                    tuple_history.append((user_msg, assistant_msg))
                    i += 1
                else:
                    tuple_history.append((user_msg, None))
            else:
                i += 1
        history = tuple_history
    else:
        history = history or []

    text = message.lower()
    try:
        if image is not None and "maverick" in text:
            response = await generate_llama_maverick_with_image(message, image)
            new_history = history + [(message, response)]
            return convert_to_messages_format(new_history)

        if any(k in text for k in ["imagen", "foto", "pintura", "dibuja"]):
            # Usa params optimizados para CPU
            img = await generate_image(message, num_inference_steps=25, width=512, height=512)
            new_history = history + [(message, ("🖼 Imagen generada:", img))]
            return convert_to_messages_format(new_history)

        if any(k in text for k in ["código", "programa", "script", "python"]):
            code = await generate_code(message)
            code_response = f"```python\n{code}\n```"
            new_history = history + [(message, code_response)]
            return convert_to_messages_format(new_history)

        if "qwen" in text:
            resp = await generate_sambanova(message, MODELS["qwen"])
            new_history = history + [(message, resp)]
            return convert_to_messages_format(new_history)

        if "meta-llama" in text or "llama 3.1" in text:
            resp = await generate_sambanova(message, MODELS["llama_meta"])
            new_history = history + [(message, resp)]
            return convert_to_messages_format(new_history)

        if "weather" in text or "clima" in text:
            location = text.split("in")[-1].strip() if "in" in text else "Madrid"  # Default location
            weather_resp = await get_current_weather(location)
            weather_str = f"Clima en {weather_resp['location']}: {weather_resp['temperature']}° {weather_resp['unit']}"
            new_history = history + [(message, weather_str)]
            return convert_to_messages_format(new_history)

        resp = await generate_sambanova(message, MODELS["chat"])
        new_history = history + [(message, resp)]
        return convert_to_messages_format(new_history)

    except Exception as e:
        error_msg = f"❌ Error: {str(e)}"
        new_history = history + [(message, error_msg)]
        return convert_to_messages_format(new_history)


with gr.Blocks(title="Asistente IA Multimodal") as app:
    chatbot = gr.Chatbot(height=500, type="messages")
    user_input = gr.Textbox(label="Escribe tu mensaje", placeholder="Escribe aquí tu consulta...")
    image_input = gr.Image(type="pil", label="Imagen (opcional para análisis con Maverick)")
    submit_btn = gr.Button("Enviar", variant="primary")

    # Evento para submit (usa asyncio para async)
    def submit_handler(message, image, history):
        if message:
            future = asyncio.run_coroutine_threadsafe(process_input(message, image, history), asyncio.get_event_loop())
            # Gradio maneja el future automáticamente en versiones recientes
            return future
        return history

    submit_btn.click(
        fn=submit_handler,
        inputs=[user_input, image_input, chatbot],
        outputs=chatbot
    )
    user_input.submit(
        fn=submit_handler,
        inputs=[user_input, image_input, chatbot],
        outputs=chatbot
    )

    # Limpiar inputs después de enviar
    def clear_inputs():
        return "", None

    submit_btn.click(clear_inputs, outputs=[user_input, image_input])


if __name__ == "__main__":
    print("🚀 Aplicación iniciada en http://localhost:7862")
    # Precalienta el modelo de imagen al inicio
    asyncio.run(generate_image("dummy", num_inference_steps=1))
    app.launch(server_port=7862, debug=True, share=False)

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access '__iter__' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.__iter__'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access '__iter__' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.__iter__'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access '__iter__' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.__iter__'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribu

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access '__iter__' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.__iter__'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribute is deprecated. Please access '__iter__' over 'StableDiffusionPipeline's config object instead, e.g. 'scheduler.config.__iter__'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
/usr/local/lib/python3.12/dist-packages/diffusers/configuration_utils.py:141: FutureWarning: Accessing config attribute `__iter__` directly via 'StableDiffusionPipeline' object attribu

SyntaxError: no binding for nonlocal 'generate_image' found (ipython-input-3442818507.py, line 106)